In [1]:
# load all test_articles in
import os

test_article_dir = "../test_articles"

articles_map: dict[str, str] = {}

for article_path in os.listdir(test_article_dir):
    full_article_path = os.path.join(test_article_dir, article_path)
    with open(full_article_path, "r", encoding="utf-8") as file:
        articles_map[article_path[:-3]] = file.read()

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

title_vectors: dict[str, np.array] = {}
for i, title in enumerate(articles_map.keys()):
    embeddings = model.encode(title)
    title_vectors[title] = np.array(embeddings)
    print(f"{i / len(articles_map) * 100:.2f}%", end="\r")
print("Finished.")

Finished.


In [ ]:
def tokenize(sentence: str) -> list[str]:
    return sentence.replace(".", "").replace("!", "").replace("?", "").replace(",", "").split(" ")

article_backlinks: dict[str, list[str]]
all_titles = list(articles_map.keys())
title_tokenizations = {t: tokenize(t) for t in all_titles}

def parse_article(title: str, content: str):
    words = tokenize(content)

    # find 1 word matches
    for word in words:
        for title in all_titles:
            

    # find 2 word matches
    for word, next_word in zip(words, words[1:]):
        query = f"{word} {next_word}"


title = list(articles_map.keys())[0]
content = articles_map[title]

parse_article(title, content)